In [14]:
import pandas as pd
import networkx as net
import numpy as num
from random import random,randint 

In [65]:
n=100
k=0
world = num.zeros(n)
weights = [random()-0.5 for _ in range(n)]
print(world,'\n',weights)

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.] 
 [-0.24519129428450592, 0.4411123373329393, -0.29134821055728843, -0.07577024777951147, -0.44451375112633595, 0.33573992613848, -0.06111002083868056, 0.4422154841613487, 0.03700506122415015, -0.3894364267036813, -0.22522048591386368, -0.36143090959561586, 0.1515132158523208, 0.04277705283053801, -0.47043796282100614, -0.11023050190072381, -0.2750480666270888, 0.27102925990147697, 0.13945797575886887, 0.10261839804770279, 0.0032028341538545835, -0.3361004354041882, 0.1285352735988219, -0.36044461571213837, 0.13837791143413958, -0.3394272691199146, 0.41024704611181606, 0.1851612429208515

In [49]:
def utility(world, weights,n):
    return(sum([world[i]*weights[i] for i in range(n)]))

def utility_k(world,weights,n,k=2):
    out = 0
    for i in range(n):
        z=world[i]*weights[i]
        if i==0:
            zk=world[-1]*weights[-1]+world[i+1]*weights[i+1]
        elif i==n-1:
            zk=world[i-1]*weights[i-1]+world[0]*weights[0]
        else:
            zk = world[i-1]*weights[i-1]+world[i+1]*weights[i+1]
        out+=zk
    return out

In [37]:
def step(world):
    i=randint(0,len(world)-1)
    w2=world.copy()
    if world[i]:
        w2[i]=0
    else:
        w2[i]=1
    return w2

In [70]:
data = pd.DataFrame()
world = num.zeros(n)
temperature =1
for _ in range(1000):
    u1 = utility_k(world,weights,n)
    world2 = step(world)
    u2 = utility_k(world2,weights,n)
    if u2 > u1:
        world=world2
    elif random()<temperature:
        world=world2
    
    data=data.append({'t':temperature,'u':utility_k(world,weights,n)},ignore_index=True)
    temperature = temperature-temperature*0.01


In [71]:
data
    

,t,u
0,1.000000,-0.366754
1,0.990000,-0.909572
2,0.980100,-1.492268
3,0.970299,-1.863578
4,0.960596,-1.555758
5,0.950990,-1.099223
6,0.941480,-0.660342
7,0.932065,-1.119676
8,0.922745,-0.425244
9,0.913517,0.462720


In [63]:
ax.show()

AttributeError: 'AxesSubplot' object has no attribute 'show'